In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from scipy.stats import binom
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# Estilo dos sliders
# ==========================================
slider_style = {'description_width': 'initial'}
slider_layout = widgets.Layout(width='600px', margin='10px auto')

# ==========================================
# Sliders
# ==========================================
prob_slider = widgets.FloatSlider(
    value=0.88, min=0.60, max=1.0, step=0.01,
    description='Chance de Comparecer:',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

seats_sold_var = widgets.IntSlider(
    value=130, min=120, max=200, step=1,
    description='Assentos Vendidos:',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

risk_accepted = widgets.FloatSlider(
    value=0.07, min=0.01, max=0.5, step=0.01,
    description='Risco Aceito:',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

vagas_slider = widgets.IntSlider(
    value=100, min=10, max=1000, step=10,
    description='Vagas Vendidas a Mais (Ano):',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

receita_slider = widgets.IntSlider(
    value=500, min=100, max=5000, step=50,
    description='Receita Média por Vaga (R$):',
    continuous_update=True,
    style=slider_style,
    layout=slider_layout
)

# ==========================================
# Outputs separados
# ==========================================
output_overbooking = widgets.Output()
output_simulacao = widgets.Output()

# ==========================================
# Função: Atualiza Overbooking
# ==========================================
def update_overbooking(*args):
    with output_overbooking:
        clear_output(wait=True)

        p = prob_slider.value
        s = seats_sold_var.value
        d = 120
        r = risk_accepted.value

        seats_sold = np.arange(110, s+1)
        probability = 1 - binom.cdf(d, seats_sold, p)

        # Gráfico de Overbooking
        fig, ax = plt.subplots(figsize=(14, 6))
        ax.plot(seats_sold, probability, linewidth=2, color='blue')
        ax.axhline(y=r, color='red', linestyle='--', linewidth=1)
        ax.set_xlabel('Assentos Vendidos', fontsize=12)
        ax.set_ylabel('Probabilidade de Overbooking', fontsize=12)
        ax.set_title('Risco de Overbooking para 120 Passageiros', fontsize=16)
        ax.grid(alpha=0.3)
        plt.show()

        # Tabela de probabilidades
        table = pd.DataFrame({'Assentos Vendidos': seats_sold, 'Probabilidade de Overbooking': probability})
        table = table.iloc[10:21, :]  # Linha 10 até 20
        pd.set_option('display.float_format', '{:.4f}'.format)
        display(table)

        if not table.empty:
            tab = table.iloc[-1, :]
            print(f"\nA probabilidade de vender {int(tab['Assentos Vendidos'])} passagens e ocorrer overbooking é de {round(tab['Probabilidade de Overbooking']*100, 2)}%.")

# ==========================================
# Função: Atualiza Simulação de Lucro
# ==========================================
def update_simulacao(*args):
    with output_simulacao:
        clear_output(wait=True)

        # Parametros
        prob_sucesso = prob_slider.value
        vagas_ano = vagas_slider.value
        receita_por_vaga = receita_slider.value

        custo_inicial = 50000
        custo_operacional_anual = 10000
        sims = 1000
        meses = np.arange(1, 13)

        vagas_mes = vagas_ano // 12

        sim_mensal = np.random.binomial(vagas_mes, prob_sucesso, size=(sims, 12)) * receita_por_vaga
        sim_mensal[:, 0] -= custo_inicial
        sim_mensal[:, -1] -= custo_operacional_anual

        lucro_mensal_medio = sim_mensal.mean(axis=0)
        lucro_total_simulado = sim_mensal.sum(axis=1)

        # Gráficos
        fig, axs = plt.subplots(2, 2, figsize=(18, 10))
        fig.suptitle('Análise de Lucros - Vários Cenários', fontsize=20, color="#333333")

        cenarios = {
            'Real (Média)': lucro_mensal_medio,
            'Pessimista (-1σ)': lucro_mensal_medio - sim_mensal.std(axis=0),
            'Bom (+1σ)': lucro_mensal_medio + sim_mensal.std(axis=0),
            'Otimista (95º percentil)': np.percentile(sim_mensal, 95, axis=0)
        }

        cores = ['royalblue', 'tomato', 'mediumseagreen', 'gold']

        for ax, (title, valores), cor in zip(axs.flatten(), cenarios.items(), cores):
            ax.bar(meses, valores, color=cor)
            ax.set_title(title, fontsize=14)
            ax.axhline(0, color='black', linestyle='--')
            ax.grid(alpha=0.3)
            ax.set_xticks(meses)
            ax.set_xlabel('Mês')
            ax.set_ylabel('Lucro Líquido (R$)')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

        # Resumo
        lucro_real = lucro_total_simulado.mean()
        lucro_pessimista = lucro_real - lucro_total_simulado.std()
        lucro_bom = lucro_real + lucro_total_simulado.std()
        lucro_otimista = np.percentile(lucro_total_simulado, 95)
        prob_abaixo_50k = (lucro_total_simulado < 50000).mean() * 100
        melhor_mes = np.argmax(lucro_mensal_medio) + 1
        pior_mes = np.argmin(lucro_mensal_medio) + 1

        resumo_html = f"""
        <div style="border:2px solid #4CAF50; padding:20px; border-radius:10px; font-family: Arial; max-width:800px; margin:auto;">
            <h2 style="color:#4CAF50; text-align:center;">Resumo Financeiro</h2>
            <ul>
                <li><b>Lucro Líquido Médio Anual:</b> R$ {lucro_real:.2f}</li>
                <li><b>ROI Médio:</b> {((lucro_real/50000)*100):.2f}%</li>
                <li><b>Probabilidade de Receita Abaixo de R$50.000:</b> {prob_abaixo_50k:.2f}%</li>
            </ul>
            <h3 style="color:#4CAF50;">Cenários Simulados:</h3>
            <ul>
                <li><b>Pessimista:</b> R$ {lucro_pessimista:.2f}</li>
                <li><b>Real:</b> R$ {lucro_real:.2f}</li>
                <li><b>Bom:</b> R$ {lucro_bom:.2f}</li>
                <li><b>Otimista:</b> R$ {lucro_otimista:.2f}</li>
            </ul>
            <h3 style="color:#4CAF50;">Destaques Mensais:</h3>
            <ul>
                <li><b>Melhor mês:</b> Mês {melhor_mes}</li>
                <li><b>Pior mês:</b> Mês {pior_mes}</li>
            </ul>
        </div>
        """
        display(HTML(resumo_html))

# ==========================================
# Ligar sliders às duas funções
# ==========================================
for slider in [prob_slider, seats_sold_var, risk_accepted]:
    slider.observe(update_overbooking, names='value')

for slider in [prob_slider, vagas_slider, receita_slider]:
    slider.observe(update_simulacao, names='value')

# ==========================================
# Interface final
# ==========================================
controls = widgets.VBox([
    prob_slider, seats_sold_var, risk_accepted, 
    vagas_slider, receita_slider
], layout=widgets.Layout(align_items='center'))

display(controls, output_overbooking, output_simulacao)

# Atualização inicial
update_overbooking()
update_simulacao()


Output()

Output()